# **Conexão com API e captura dos dados**

In [ ]:
import requests
import pandas as pd
from google.colab import auth
from google.cloud import storage
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from datetime import datetime

# Função para configurar a sessão de requisição com retry e timeout
def configurar_sessao():
    session = requests.Session()
    retry = Retry(
        total=5,  # Número de tentativas
        backoff_factor=1,  # Tempo de espera entre as tentativas (exponencial)
        status_forcelist=[429, 500, 502, 503, 504],  # Códigos HTTP para retry
        allowed_methods=["GET"],  # Métodos HTTP a aplicar retry
    )
    adapter = HTTPAdapter(max_retries=retry)
    session.mount("https://", adapter)
    return session

# Configurar a sessão de requisição
sessao = configurar_sessao()

# URL para obter a lista de todos os deputados
url_deputados = "https://dadosabertos.camara.leg.br/api/v2/deputados"

# Parâmetros para a lista de deputados
params_deputados = {
    'itens': 100,
    'pagina': 1
}

# Função para capturar despesas de um deputado específico
def obter_despesas_deputado(deputado_id):
    url_despesas = f"https://dadosabertos.camara.leg.br/api/v2/deputados/{deputado_id}/despesas"
    params_despesas = {
        'ano': [2024],
        'itens': 100,
        'pagina': 1,
        'ordem': 'desc',
        'ordenarPor': 'dataDocumento'
    }
    despesas = []
    while True:
        try:
            response = sessao.get(url_despesas, params=params_despesas, timeout=10)
            if response.status_code == 200:
                data = response.json()['dados']
                if not data:
                    break
                for item in data:
                    item['idDeputado'] = deputado_id  # Adicionar ID do deputado em cada item
                despesas.extend(data)
                params_despesas['pagina'] += 1
            else:
                print(f"Erro ao obter despesas do deputado {deputado_id}: {response.status_code}")
                break
        except requests.exceptions.Timeout:
            print(f"Timeout na requisição para o deputado {deputado_id}. Tentando novamente...")
    return despesas

# Função para capturar informações adicionais dos deputados
def obter_info_deputado(deputado_id):
    url_info = f"https://dadosabertos.camara.leg.br/api/v2/deputados/{deputado_id}"
    try:
        response = sessao.get(url_info, timeout=10)
        if response.status_code == 200:
            data = response.json()['dados']
            info = {
                'id': data['id'],
                'nome': data['ultimoStatus']['nome'],
                'partido': data['ultimoStatus']['siglaPartido'],
                'estado': data['ultimoStatus']['siglaUf']
            }
            return info
        else:
            print(f"Erro ao obter info do deputado {deputado_id}: {response.status_code}")
            return None
    except requests.exceptions.Timeout:
        print(f"Timeout ao obter info do deputado {deputado_id}.")
        return None

# Função para capturar a lista de deputados
def obter_lista_deputados():
    deputados = []
    while True:
        try:
            response = sessao.get(url_deputados, params=params_deputados, timeout=10)
            if response.status_code == 200:
                data = response.json()['dados']
                if not data:
                    break
                deputados.extend(data)
                params_deputados['pagina'] += 1
            else:
                print(f"Erro ao obter lista de deputados: {response.status_code}")
                break
        except requests.exceptions.Timeout:
            print("Timeout ao obter lista de deputados. Tentando novamente...")
    return deputados

# Obter a lista completa de deputados
lista_deputados = obter_lista_deputados()

# Inicializar lista para armazenar todas as despesas
todas_despesas = []
deputados_info = []

# Iterar sobre cada deputado e capturar suas despesas e informações
for deputado in lista_deputados:
    deputado_id = deputado['id']
    nome_deputado = deputado['nome']

    print(f"Capturando despesas do deputado: {nome_deputado} (ID: {deputado_id})")

    # Obter despesas do deputado
    despesas_deputado = obter_despesas_deputado(deputado_id)
    todas_despesas.extend(despesas_deputado)

    # Obter informações adicionais do deputado
    info_deputado = obter_info_deputado(deputado_id)
    if info_deputado:
        deputados_info.append(info_deputado)

# Converter a lista de despesas e informações em DataFrames
df_despesas = pd.DataFrame(todas_despesas)
df_info_deputados = pd.DataFrame(deputados_info)

# Verifique as colunas de df_despesas
print(df_despesas.columns)

# Juntar as informações de deputados com as despesas
df_completo = pd.merge(df_despesas, df_info_deputados, left_on='idDeputado', right_on='id', how='left')

# Exibir as primeiras linhas do DataFrame para visualização
print(df_completo.head())

# Obter a data atual para incluir no nome do arquivo
data_atual = datetime.now().strftime("%Y-%m-%d")

# Definir o nome do arquivo com a data atual
file_name = f'despesas_deputados_{data_atual}.csv'

# Salvar o DataFrame como CSV no Google Cloud Storage
auth.authenticate_user()

# Criar o cliente do Google Cloud Storage
client = storage.Client()

# Defina o nome do bucket e o arquivo que será salvo
bucket_name = 'camara_deputados'

# Referenciar o bucket e o arquivo
bucket = client.bucket(bucket_name)
blob = bucket.blob(file_name)

# Salvar o CSV no bucket
df_completo.to_csv(file_name, index=False)
blob.upload_from_filename(file_name)

print(f"Arquivo salvo com sucesso no bucket {bucket_name} como {file_name}.")


Capturando despesas do deputado: Abilio Brunini (ID: 220593)
Capturando despesas do deputado: Acácio Favacho (ID: 204379)
Capturando despesas do deputado: Adail Filho (ID: 220714)
Capturando despesas do deputado: Adilson Barroso (ID: 221328)
Capturando despesas do deputado: Adolfo Viana (ID: 204560)
Capturando despesas do deputado: Adriana Ventura (ID: 204528)
Capturando despesas do deputado: Adriano do Baldy (ID: 121948)
Capturando despesas do deputado: Aécio Neves (ID: 74646)
Capturando despesas do deputado: Afonso Hamm (ID: 136811)
Capturando despesas do deputado: Afonso Motta (ID: 178835)
Capturando despesas do deputado: Aguinaldo Ribeiro (ID: 160527)
Capturando despesas do deputado: Airton Faleiro (ID: 204495)
Capturando despesas do deputado: AJ Albuquerque (ID: 204549)
Capturando despesas do deputado: Alberto Fraga (ID: 73579)
Capturando despesas do deputado: Alberto Mourão (ID: 74696)
Capturando despesas do deputado: Albuquerque (ID: 220538)
Capturando despesas do deputado: Alce

In [ ]:
import pandas as pd

# Carregar o arquivo CSV salvo
df_completo = pd.read_csv('despesas_deputados_2024-09-16.csv')  # Substitua com o nome correto do arquivo

# Exibir todas as colunas e seus tipos de dados
print("Colunas e tipos de dados do arquivo CSV:")
print(df_completo.dtypes)


Colunas e tipos de dados do arquivo CSV:
ano                    int64
mes                    int64
tipoDespesa           object
codDocumento           int64
tipoDocumento         object
codTipoDocumento       int64
dataDocumento         object
numDocumento          object
valorDocumento       float64
urlDocumento          object
nomeFornecedor        object
cnpjCpfFornecedor    float64
valorLiquido         float64
valorGlosa           float64
numRessarcimento     float64
codLote                int64
parcela                int64
idDeputado             int64
id                     int64
nome                  object
partido               object
estado                object
dtype: object


# **Carregando dados no Data Lake**

In [ ]:
from google.colab import auth
auth.authenticate_user()


In [ ]:
!pip install --upgrade pandas-gbq google-cloud-bigquery


In [ ]:
import pandas as pd
from google.cloud import bigquery
from google.colab import auth
import pandas_gbq

# Autenticar no Google
auth.authenticate_user()

# Carregar o arquivo CSV
df = pd.read_csv('despesas_deputados_2024-09-16.csv')

# Definir o ID do projeto no BigQuery
project_id = 'governoaberto'
dataset_id = 'camara_deputados'
table_id = 'despesas_deputados'

# Definir o esquema (schema) da tabela no BigQuery
schema = [
    bigquery.SchemaField("ano", "INTEGER"),
    bigquery.SchemaField("cnpjCpfFornecedor", "STRING"),
    bigquery.SchemaField("codDocumento", "INTEGER"),
    bigquery.SchemaField("codLote", "INTEGER"),
    bigquery.SchemaField("codTipoDocumento", "INTEGER"),
    bigquery.SchemaField("dataDocumento", "STRING"),  # Pode mudar para DATE se o formato for adequado
    bigquery.SchemaField("mes", "INTEGER"),
    bigquery.SchemaField("nomeFornecedor", "STRING"),
    bigquery.SchemaField("numDocumento", "STRING"),
    bigquery.SchemaField("numRessarcimento", "STRING"),
    bigquery.SchemaField("parcela", "INTEGER"),
    bigquery.SchemaField("tipoDespesa", "STRING"),
    bigquery.SchemaField("tipoDocumento", "STRING"),
    bigquery.SchemaField("urlDocumento", "STRING"),
    bigquery.SchemaField("valorDocumento", "FLOAT"),
    bigquery.SchemaField("valorGlosa", "FLOAT"),
    bigquery.SchemaField("valorLiquido", "FLOAT"),
    bigquery.SchemaField("idDeputado", "INTEGER"),
    bigquery.SchemaField("id", "INTEGER"),
    bigquery.SchemaField("nome", "STRING"),
    bigquery.SchemaField("partido", "STRING"),
    bigquery.SchemaField("estado", "STRING")]

# Criar cliente BigQuery
client = bigquery.Client(project=project_id)

# Definir a referência da tabela
table_ref = client.dataset(dataset_id).table(table_id)

# Definir a configuração da tabela
table = bigquery.Table(table_ref, schema=schema)

# Criar a tabela no BigQuery
table = client.create_table(table)  # Se já existir, será necessário remover este passo

print(f"Tabela {table_id} criada com sucesso.")

# Agora, carregar o DataFrame para a tabela recém-criada
pandas_gbq.to_gbq(df, f'{dataset_id}.{table_id}', project_id=project_id, if_exists='replace')

print(f"Dados inseridos na tabela {table_id} com sucesso.")


Tabela despesas_deputados criada com sucesso.


100%|██████████| 1/1 [00:00<00:00, 681.56it/s]

Dados inseridos na tabela despesas_deputados com sucesso.
